## 서울시 자전거 따릉이 데이터 전처리
* 데이터 출처 : [데이터셋> 데이터 이용하기 | 서울열린데이터광장](http://data.seoul.go.kr/dataList/datasetView.do?infId=OA-15182&srvType=F&serviceKind=1&currentPageNo=1)
* 전체 데이터를 가져와서 연도별로 병합(concat)을 해주면 1G가 넘는 용량으로 파일이 만들어 집니다.
* 1G가 넘는 파일을 불러와서 작업을 하려면 장비에 부담이 되기 때문에 데이터를 줄여줍니다.

In [1]:
import pandas as pd

In [2]:
%ls data/seoul-bike-*

data/seoul-bike-2016-reduce.csv  data/seoul-bike-2018-reduce.csv
data/seoul-bike-2016.csv         data/seoul-bike-2018.csv
data/seoul-bike-2017-reduce.csv  data/seoul-bike-2019-reduce.csv
data/seoul-bike-2017.csv         data/seoul-bike-2019.csv


In [3]:
year = 2019

In [4]:
%time df = pd.read_csv(f"data/seoul-bike-{year}.csv", low_memory=False)
df.shape

CPU times: user 19.1 s, sys: 2.81 s, total: 21.9 s
Wall time: 23.5 s


(5471294, 11)

In [5]:
df.columns

Index(['자전거번호', '대여일시', '대여대여소번호', '대여대여소명', '대여거치대', '반납일시', '반납대여소번호',
       '반납대여소명', '반납거치대', '이용시간(분)', '이용거리(M)'],
      dtype='object')

In [ ]:
%time df["자전거번호SPB"] = df["자전거번호"].apply(lambda x : x.split("-")[1])

In [ ]:
df.info()

In [12]:
# 반납대여소번호와 반납대여소명은 다르게 입력된 데이터가 많이 있다.(데이터 제공부서에 문의가 필요하다.)
# 분석할 때는 대여소번호를 기준으로 분석하도록 한다.
df.head()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-10957,2019-01-01 00:02:16,1408,1408. 먹골역 6번출구 앞,3,2019-01-01 00:07:07,1433,1408. 먹골역 6번출구 앞,4,4,1020.0
1,SPB-17170,2019-01-01 00:03:09,615,615. 용두동 레미안허브리츠아파트 앞,4,2019-01-01 00:07:27,612,615. 용두동 레미안허브리츠아파트 앞,14,3,700.0
2,SPB-14130,2019-01-01 00:01:44,1177,1177. 수명중?고교,7,2019-01-01 00:08:09,1152,1177. 수명중?고교,1,5,1090.0
3,SPB-15414,2019-01-01 00:02:28,646,646. 장한평역 1번출구 (국민은행앞),3,2019-01-01 00:08:30,648,646. 장한평역 1번출구 (국민은행앞),2,5,1120.0
4,SPB-08668,2019-01-01 00:04:14,2273,2273. 일동제약 사거리,5,2019-01-01 00:08:54,2255,2273. 일동제약 사거리,25,4,930.0


In [9]:
df.tail()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
5471289,SPB-16314,2019-05-31 23:25:34,2217,아크로리버뷰 부지 앞,14,2019-06-01 08:49:59,2289,남태령역 2번출구,4,564,5060.0
5471290,SPB-05369,2019-05-31 22:12:06,722,LG전자베스트샵 신정점,1,2019-06-01 09:37:07,1140,목동사거리 버스정류장,2,684,1790.0
5471291,SPB-14277,2019-05-31 21:48:05,1814,두산위브아파트 옆 상가건물 앞,8,2019-06-01 09:57:32,1849,대륭포스트타워5차,1,729,1580.0
5471292,SPB-10222,2019-05-31 21:19:40,1295,잠실역 8번출구,20,2019-06-01 10:04:11,626,군자교 서측 녹지대,7,253,6980.0
5471293,SPB-12224,2019-05-31 09:19:29,567,성수역 2번출구 앞,7,2019-06-01 12:38:00,567,성수역 2번출구 앞,7,235,760.0


In [ ]:
# 데이터가 너무 커서 로드하는데 시간이 오래 걸리거나 전처리에 시간이 오래 걸린다.
# 자원을 효율적으로 사용하기 위해 용량이 큰 텍스트 데이터를 빼주고 나중에 join이나 merge를 통해 불러오도록 한다.
df_reduce = df[['자전거번호SPB', '대여일시', '대여대여소번호', '반납일시', '반납대여소번호',
       '이용시간(분)', '이용거리(M)']]
df_reduce.shape

In [ ]:
df_reduce.head()

In [ ]:
%time df_reduce.to_csv(f"data/seoul-bike-{year}-reduce.csv", index=False)

In [ ]:
%time df_reduce_file = pd.read_csv(f"data/seoul-bike-{year}-reduce.csv", low_memory=False)
df_reduce_file.shape

In [ ]:
# 용량이 절반정도 줄어 들었다.
%ls -lah data/seoul-bike-*

## 대여소 정보를 별도의 CSV 파일로 만들기
* 파일 용량을 위해 대여소 정보를 제외했다.
* 크롤링으로 가져온 대여소 정보도 있지만 여기에 있는 정보로 대여소 파일을 간단하게 만들어 본다.

In [ ]:
df["대여소명"] = df["대여대여소명"].apply(lambda x : x.split(".")[-1].strip())

In [ ]:
df_station = df.drop_duplicates(["대여대여소번호", "대여소명"]
    keep='last').sort_values(by="대여대여소번호")
df_station.columns = ["대여소번호", "대여소명"]
print(df_station.shape)
df_station.head()

In [ ]:
# 중복된 대여소가 있는지 확인한다. 
df_station["대여소번호"].value_counts().head()

In [ ]:
# 제대로 제거가 되었는지 확인
df_station[df_station["대여소번호"] == 1042]

In [ ]:
print(df_station.shape)
df_station.drop_duplicates(["대여소번호"], keep='last', inplace=True)
print(df_station.shape)

In [ ]:
df_station = df_station[["대여소번호", "대여소명"]].reset_index(drop=True)

In [ ]:
# 제대로 제거가 되었는지 확인
df_station[df_station["대여소번호"] == 1042]

In [ ]:
# csv 파일로 저장
df_station.to_csv("data/bike-station-no-name.csv", index=False)

In [ ]:
# csv 파일이 제대로 저장되었는지 확인
pd.read_csv("data/bike-station-no-name.csv").head()